<a href="https://colab.research.google.com/github/alex-jk/SWB-GVCEH/blob/main/models/relevance_model/Reddit_Relevancy_Models_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install and import necessary libraries

In [1]:
!pip install setfit
!pip install tqdm
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import gdown
import json
import pandas as pd
import os

Mounted at /content/drive


In [3]:
from setfit import SetFitModel
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")

from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import concurrent.futures

import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

model_sent_transformer = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Import the full Reddit dataset
- remove duplicates to make sure that posts are unqiue
- TitleText is the text column of interest

In [5]:
file_path = '/content/drive/My Drive/SWB-GVCEH/Complete_Data_v3.json'
reddit_data_df = pd.read_json(file_path)

print(f"\ncd_test shape: {reddit_data_df.shape}")
print(f"\n--------------- Columns: {reddit_data_df.columns}")

select_cols = ['Subreddit', 'Title', 'Text', 'TitleText']
reddit_data_df = reddit_data_df[select_cols]
reddit_data_df.drop_duplicates(inplace=True)
reddit_data_df.reset_index(drop=True, inplace=True)

print("\n------------------------------")
print(reddit_data_df.head())


cd_test shape: (11160, 23)

--------------- Columns: Index(['index', 'Subreddit', 'Title', 'Text', 'TitleText', 'relevance_score',
       'most_common_centroid_id', 'top_terms_from_centroid',
       'topics_from_centroid', 'Score_model2', 'label_model2', 'label_model1',
       'relevant_sentences', 'topic_num', 'Relevant_document',
       'Relevant_topic', 'topic_label', 'Sentiment_Full',
       'Sentence_Level_Sentiment_Compund',
       'Relevent_Sentence_Sentiment_Compund', 'BERT_sentiment_all',
       'BERT_sentiments_relevant_sentences', 'manual_label'],
      dtype='object')

------------------------------
    Subreddit                                              Title  \
0  VictoriaBC  True change around homelessness from the homel...   
1  VictoriaBC  New transitional housing facility on Douglas o...   
2  VictoriaBC   Where to buy a reasonably priced Christmas tree?   
3  VictoriaBC  Looking for a Christmas tree? Please consider ...   
4  VictoriaBC                           

#### Reddit Data Labelling - Model #1

#### Load Twitter Cluster Centroids

In [9]:
file_path_twitter_centroids = "/content/drive/My Drive/SWB-GVCEH/twitter_cluster_centroids.json"

# Read the JSON file
with open(file_path_twitter_centroids, "r") as f:
    centroids_list = json.load(f)

cluster_centroids = np.array(centroids_list)
print(cluster_centroids)

[[ 0.07606746 -0.03684408  0.07050317 ... -0.04477242 -0.05458607
   0.06265755]
 [ 0.03922427 -0.00116647  0.05264552 ... -0.03459585 -0.04934484
   0.0509878 ]
 [ 0.08159573 -0.00581049  0.06172452 ... -0.05318972 -0.0436865
   0.03083117]
 ...
 [ 0.04692682  0.04268746  0.03507463 ... -0.02552383 -0.04925404
   0.03933679]
 [ 0.01194589  0.00625541  0.04830575 ... -0.04271128 -0.0092378
   0.0193102 ]
 [ 0.01649493  0.01140771  0.01363297 ... -0.05939158  0.01116832
   0.01196186]]


##### Functions to process reddit posts

In [10]:
# Function to split text into sentences
def spacy_sentence_split(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

# Function to process a single Reddit post and determine its relevance
def process_post_for_relevance(post, model, centroids, threshold):
    sentences = spacy_sentence_split(post)
    sentence_embeddings = model.encode(sentences, show_progress_bar=False)
    sentence_relevance_scores = cosine_similarity(sentence_embeddings, centroids)
    max_scores = sentence_relevance_scores.max(axis=1)
    max_score_indices = sentence_relevance_scores.argmax(axis=1)
    # print(len(sentence_embeddings))
    # print(max_scores)
    # print(max_score_indices)

    relevant_sentences = max_scores > threshold
    relevant_centroid_ids = max_score_indices[relevant_sentences]
    # print(relevant_centroid_ids)
    relevance_score = relevant_sentences.mean()  # Proportion of sentences above the threshold

    centroid_counter = Counter(relevant_centroid_ids)
    # print(centroid_counter)
    if len(centroid_counter) == 0:
        most_common_centroid_ids = None  # No relevant sentences
    else:
        # Find the maximum count for the centroids
        max_count = centroid_counter.most_common(1)[0][1]
        # Get all centroid IDs that have the maximum count (ties included)
        most_common_centroid_ids = [centroid_id for centroid_id, count in centroid_counter.items() if count == max_count]

    return relevance_score, most_common_centroid_ids

In [11]:
# Parallel processing function
def parallel_relevance_computation(reddit_posts, model, centroids, threshold, max_workers=10):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda post: process_post_for_relevance(post, model, centroids, threshold), reddit_posts), total=len(reddit_posts)))

    # Unpack the results into two separate lists
    relevance_scores, most_common_centroid_ids = zip(*results)
    return relevance_scores, most_common_centroid_ids

##### Assign Model #1 labels

In [12]:
def apply_model01_labeling(dataframe, text_column, model, centroids, threshold_score=0.5, label_threshold=0.1):
    """
    Computes relevance scores and centroid IDs for a given DataFrame, then assigns labels based on relevance.

    Parameters:
    - dataframe: pd.DataFrame, the DataFrame to process.
    - titles_column: str, the name of the column containing text to compute relevance for.
    - model: The model used to transform text into embeddings.
    - centroids: np.ndarray, the cluster centroids used for computing relevance.
    - threshold_score: float, the threshold used within the parallel relevance computation.
    - label_threshold: float, the threshold above which the relevance score is considered significant for labeling.

    The function updates the input DataFrame with three new columns: 'relevance_score',
    'most_common_centroid_id', and 'label_model1'.
    """
    # Assuming the implementation of parallel_relevance_computation is available in your environment
    relevance_scores, most_common_centroid_ids = parallel_relevance_computation(
        dataframe[text_column].tolist(), model, centroids, threshold=threshold_score)

    # Assign the computed values to the DataFrame
    dataframe['relevance_score'] = relevance_scores
    dataframe['most_common_centroid_id'] = most_common_centroid_ids

    # Assign labels based on the relevance score threshold
    dataframe['label_model1'] = np.where(dataframe['relevance_score'] >= label_threshold, 1, 0)


In [13]:
# Filter relevant posts based on a relevance score threshold
# reddit_data_df['label_model1'] = np.where(reddit_data_df['relevance_score'] >= 0.1, 1, 0)
apply_model01_labeling(reddit_data_df, 'TitleText', model_sent_transformer, cluster_centroids, 0.5, 0.1)

relevant_reddit_posts = reddit_data_df[reddit_data_df['relevance_score'] > 0.1].reset_index(drop=True)
print(f"\nOriginal number of posts: {len(reddit_data_df.index)}")
print(f"\nNumber of relevant posts: {len(relevant_reddit_posts.index)}")

  0%|          | 0/11151 [00:00<?, ?it/s]


Original number of posts: 11151

Number of relevant posts: 3311


#### SetFit Relevancy Model - Model #2

##### Import train and test data
Import labelled json data sample

In [6]:
file_path_sample02 = '/content/drive/My Drive/SWB-GVCEH/reddit_manual_labels_sample02.json'

reddit_manual_labels_sample02 = pd.read_json(file_path_sample02, lines=True)
print(f"\n---- df reddit_manual_labels_sample02 shape: {reddit_manual_labels_sample02.shape}")
print(f"\n Columns: {reddit_manual_labels_sample02.columns}")

print(f"\n-----------------------")
print(reddit_manual_labels_sample02.head())


---- df reddit_manual_labels_sample02 shape: (118, 11)

 Columns: Index(['Subreddit', 'Title', 'Text', 'TitleText', 'relevance_score',
       'most_common_centroid_id', 'Score_model2', 'label_model2',
       'label_model1', 'relevant_sentences', 'manual_label'],
      dtype='object')

-----------------------
         Subreddit                                              Title  Text  \
0       VictoriaBC  Victoria Womens' Flag Football League is start...  None   
1       VictoriaBC  Rowers Caileigh Filmer and Hillary Janssens wi...  None   
2  britishcolumbia  She introduced the exotic dish of pizza to CBC...  None   
3       VictoriaBC   City of Victoria Releases New Solar Rooftop Tool  None   
4            Sooke  New wrestling club launches in Sooke - Coaches...  None   

                                           TitleText  relevance_score  \
0  Victoria Womens' Flag Football League is start...              0.0   
1  Rowers Caileigh Filmer and Hillary Janssens wi...              0.

##### Split into test and train

In [7]:
# Shuffle the DataFrame
df_shuffled = reddit_manual_labels_sample02.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled['TitleText'].fillna('', inplace=True)

split_size = int(len(df_shuffled) * 0.65)

select_cols = ['TitleText', 'manual_label']
# Split the DataFrame into two parts
sample_train = df_shuffled[:split_size][select_cols].copy().reset_index(drop=True)
sample_test = df_shuffled[split_size:][select_cols].reset_index(drop=True)
print(f"\nTrain dataset length: {len(sample_train)}, test dataset length: {len(sample_test)}")

value_counts_train = sample_train['manual_label'].value_counts()
print("\n----------------")
print(value_counts_train)

value_counts_test = sample_test['manual_label'].value_counts()
print("\n----------------")
print(value_counts_test)


Train dataset length: 76, test dataset length: 42

----------------
0    52
1    24
Name: manual_label, dtype: int64

----------------
0    27
1    15
Name: manual_label, dtype: int64


In [8]:
train_dataset = Dataset.from_pandas(sample_train)
test_dataset = Dataset.from_pandas(sample_test)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [11]:
import random
import torch

def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # For some operations, you may also want to enable deterministic behavior in PyTorch
    # This may impact performance
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example usage
set_seed(42)

In [12]:
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss

model_id_reddit = "sentence-transformers/all-MiniLM-L6-v2"
model_reddit = SetFitModel.from_pretrained(model_id_reddit)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
# Configure the SetFitTrainer
trainer = SetFitTrainer(
    model=model_reddit,
    train_dataset=dataset_dict["train"],  # training dataset
    eval_dataset=dataset_dict["test"],    # test dataset
    loss_class=CosineSimilarityLoss,      # Specify the loss class
    batch_size=8,                        # Batch size for training
    num_iterations=20,                    # Number of iterations over the training dataset
    column_mapping={"TitleText": "text", "manual_label": "label"}  # Adjust column names as needed
)

<ipython-input-13-47bd27bb75f2>:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

In [14]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)

***** Running training *****
  Num unique pairs = 3040
  Batch size = 8
  Num epochs = 1
  Total optimization steps = 380


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.7380952380952381}


##### Save SetFit model

In [15]:
import joblib

joblib.dump(trainer, '/content/drive/My Drive/SWB-GVCEH/reddit-setfit-model.joblib')

['/content/drive/My Drive/SWB-GVCEH/reddit-setfit-model.joblib']

##### Predict

In [6]:
import joblib

# Load the trainer
trainer_reddit = joblib.load('/content/drive/My Drive/SWB-GVCEH/reddit-setfit-model.joblib')

In [ ]:
reddit_posts = reddit_data_df["TitleText"].tolist()

predictions = []

for post in tqdm(reddit_posts, desc="Predicting"):
    prediction = trainer_reddit.model.predict([post])  # Predict for each title individually
    predictions.append(prediction[0])  # Assuming prediction is a list with a single element

# Assign predictions back to your DataFrame
reddit_data_df['label_model2_setfit'] = predictions

Predicting:   0%|          | 0/11151 [00:00<?, ?it/s]

##### Print counts by label

In [ ]:
value_counts = reddit_data_df['label_model2_setfit'].value_counts()
print(value_counts)

##### Reddit posts samples - relevant and not relevant, as classified by SetFit

In [ ]:
select_label = 0
ind = 0

temp_df = reddit_data_df[reddit_data_df['label_model2_setfit']==select_label]